# 多类分类

In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat


In [79]:
data = loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [80]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

第一个任务是将我们的逻辑回归实现修改为完全向量化（即没有“for”循环）。这是因为向量化代码除了简洁外，还能够利用线性代数优化，并且通常比迭代代码快得多。但是，如果从练习2中看到我们的代价函数已经完全向量化实现了，所以我们可以在这里重复使用相同的实现。

# sigmoid函数


In [81]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
    

In [82]:
def cost(theta, X, y, learningRate):
    X = np.matrix(X) #把数组换成矩阵
    y = np.matrix(y)
    theta = np.matrix(theta)

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    last = np.sum(first - second)
    reg = (learningRate / len(X)) * np.sum(np.power(theta[:, 1:theta.shape[1] - 1], 2))

    return last / len(X) + reg

In [83]:
def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)

    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:, i])

        if(i == 0):
            grad[i] = np.sum(term / len(X))
        else:
            grad[i] = (np.sum(term) / len(X)) + (learningRate / len(X)) * theta[:, i]

    return grad

In [84]:
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

现在我们已经定义了代价函数和梯度函数，现在是构建分类器的时候了。 对于这个任务，我们有10个可能的类，并且由于逻辑回归只能一次在2个类之间进行分类，我们需要多类分类的策略。 在本练习中，我们的任务是实现一对一全分类方法，其中具有k个不同类的标签就有k个分类器，每个分类器在“类别 i”和“不是 i”之间决定。 我们将把分类器训练包含在一个函数中，该函数计算10个分类器中的每个分类器的最终权重，并将权重返回为k X（n + 1）数组，其中n是参数数量。

In [85]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learningRate):
    rows = X.shape[0]
    params = X.shape[1]
    
    all_theta = np.zeros((num_labels, params + 1))
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learningRate), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x

    return all_theta
     

In [86]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape


((5000, 401), (5000, 1), (401,), (10, 401))

In [87]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [88]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta


array([[-1.94179096e+00,  0.00000000e+00,  0.00000000e+00, ...,
         9.24976732e-04,  8.28875851e-08,  0.00000000e+00],
       [-3.26181460e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.86825542e-03, -3.30731076e-04,  0.00000000e+00],
       [-4.74678451e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.11049667e-05, -1.35583820e-07,  0.00000000e+00],
       ...,
       [-7.69234445e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -7.67012859e-05,  5.69902738e-06,  0.00000000e+00],
       [-4.50145942e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.96745713e-04,  2.68203339e-05,  0.00000000e+00],
       [-3.82190360e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.24210175e-04,  1.15717276e-05,  0.00000000e+00]])

In [ ]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_label = all_theta.shape[0]

    